In [1]:
import random
import re
from math import pi
import os
import shutil

In [ ]:
def find_tab (line): #uesd to find tab in the front of each line and reture the tabs to ensure the grammar correct
    i = 0
    tab = ""
    while line[i] == " ":
        tab = tab + line[i]
        i = i + 1
    return tab

def execution(pyfile_name): # execute quantum circuits in the formate of .py and catch possible errors.
    try:
        os.system('python ' + pyfile_name)
    except Exception as e:
        print("OS error:" + str(e))

In [2]:
gate_x = re.compile("prog.x")
gate_s = re.compile("prog.s")
gate_z = re.compile("prog.z")
gate_swap = re.compile("prog.swap")
gate_cx = re.compile("prog.cx")
gate_cz = re.compile("prog.cz")

def makeeuqalfiles(filein,fileout):
    flag=0       #set the flage to detect wethere there is any changed gates                     
    readfile = open(filein)
    writefile = open("euqal_temp","w")
    total_number=0
    total_operation_id = re.compile("# total number=")
    line = readfile.readline()
    wirte_new_file_name=re.compile("open" )
    while line:
        #####  read each line in the file and randomly change the gates to euqivalent gates
        if wirte_new_file_name.search(line):
            writefile.write(re.sub("startQiskit[0-9]+",fileout.split('.')[0],line))
            line = readfile.readline()
            continue
            
        if total_operation_id.search(line):
            total_number=int(re.sub("\D","",line))
#             print(total_number)
            
        if gate_s.search(line):
            if random.randint(0,5)>3:
                tab=find_tab(line)
                writefile.write(s2tt(tab,line,total_number)[0])
                flag=1
                total_number=s2tt(tab,line,total_number)[1]
            else:
                writefile.write(line)
            line = readfile.readline()
            continue

        if gate_swap.search(line):
            if random.randint(0,5)>3:
                tab=find_tab(line)
                writefile.write(swap2cx(tab,line,total_number)[0])
                flag=1
                total_number=swap2cx(tab,line,total_number)[1]
            else:
                writefile.write(line)
            line = readfile.readline()
            continue
            
        if gate_x.search(line):
            if random.randint(0,5)>2:
                tab=find_tab(line)
                op=[x2cx,x2hzh,x2ix,x2hssh]       ####randomly choose one combination of gates in the list of opearations
                idd=random.randint(0,3)
                writefile.write(op[idd](tab,line,total_number)[0])
                flag=1
                total_number=op[idd](tab,line,total_number)[1]
            else:
                writefile.write(line)
            line = readfile.readline()
            continue
            
        if gate_z.search(line):
            if random.randint(0,5)>3:
                tab=find_tab(line)
                op_1=[z2ss,z2cx]
                idd_1=random.randint(0,1)
                writefile.write(op_1[idd_1](tab,line,total_number)[0])
                flag=1
                total_number=op_1[idd_1](tab,line,total_number)[1]
            else:
                writefile.write(line)
            line = readfile.readline()
            continue 
            
            
        if gate_cx.search(line):
            if random.randint(0,5)>3:
                tab=find_tab(line)
                op_2=[cx2h,cx2cnot]
                idd_2=random.randint(0,1)
                writefile.write(op_2[idd_2](tab,line,total_number)[0])
                flag=1
                total_number=op_2[idd_2](tab,line,total_number)[1]
            else:
                writefile.write(line)
            line = readfile.readline()
            continue
            
        if gate_cz.search(line):
            if random.randint(0,5)>3:
                tab=find_tab(line)
                writefile.write(cz2h(tab,line,total_number)[0])
                flag=1
                total_number=cz2h(tab,line,total_number)[1]
            else:
                writefile.write(line)
        else:
            writefile.write(line)
        line = readfile.readline()

    readfile.close()
    writefile.close()
    
    readfile_11 = open('euqal_temp')  #######in order to make the totoal number correct ,should read the file again to change the totoal number
    writefile_11 = open(fileout,"w")
    line_11 = readfile_11.readline()
    while line_11:
        if total_operation_id.search(line_11):
            writefile_11.write("# total number="+str(total_number)+"\n")
        else:
            writefile_11.write(line_11)
        line_11 = readfile_11.readline()
    readfile_11.close()
    writefile_11.close()
    os.remove("./euqal_temp")
    
    
    if flag==0:
        return 0
    else:
        return total_number 
        
            

In [3]:
number_1=" # number=[0-9]+"
number= "\[[0-9]+\]"

###the following functions is uses to replace one gate to the euqivalent gates 


def x2hssh(tab,line,total_number):
    sentence= tab+"prog.h(input_qubit["+ re.sub("\D","",re.findall(number,line)[0])+ ']) # number='+ re.sub("\D","",re.findall(number_1,line)[0])+"\n"+\
            tab+"prog.s(input_qubit["+ re.sub("\D","",re.findall(number,line)[0])+ ']) # number='+ str(total_number+1) +"\n"+\
    tab+"prog.s(input_qubit["+ re.sub("\D","",re.findall(number,line)[0])+ ']) # number='+ str(total_number+2) +"\n"+\
    tab+"prog.h(input_qubit["+ re.sub("\D","",re.findall(number,line)[0])+ ']) # number='+ str(total_number+3) +"\n"
    return sentence,total_number+3

def z2ss(tab,line,total_number):
    sentence= tab+"prog.s(input_qubit["+ re.sub("\D","",re.findall(number,line)[0])+ ']) # number='+ re.sub("\D","",re.findall(number_1,line)[0])+"\n"+\
            tab+"prog.s(input_qubit["+ re.sub("\D","",re.findall(number,line)[0])+ ']) # number='+ str(total_number+1) +"\n"
    return sentence,total_number+1

def s2tt(tab,line,total_number):
    sentence= tab+"prog.t(input_qubit["+ re.sub("\D","",re.findall(number,line)[0])+ ']) # number='+ re.sub("\D","",re.findall(number_1,line)[0])+"\n"+\
            tab+"prog.t(input_qubit["+ re.sub("\D","",re.findall(number,line)[0])+ ']) # number='+ str(total_number+1) +"\n"
    return sentence,total_number+1

def swap2cx(tab,line,total_number):
    sentence= tab+"prog.cx(input_qubit["+ re.sub("\D","",re.findall(number,line)[0])+ '],input_qubit['+ re.sub("\D","",re.findall(number,line)[1])+']) # number='+ re.sub("\D","",re.findall(number_1,line)[0])+"\n"+\
            tab+"prog.cx(input_qubit["+ re.sub("\D","",re.findall(number,line)[0])+ '],input_qubit['+ re.sub("\D","",re.findall(number,line)[1])+']) # number='+ str(total_number+1) +"\n"+\
                tab+"prog.cx(input_qubit["+ re.sub("\D","",re.findall(number,line)[0])+ '],input_qubit['+ re.sub("\D","",re.findall(number,line)[1])+']) # number='+ str(total_number+2) +"\n"
    return sentence,total_number+2

def cx2cnot(tab,line,total_number):
    sentence=tab+"prog.cx(input_qubit["+ re.sub("\D","",re.findall(number,line)[0])+ '],input_qubit['+ re.sub("\D","",re.findall(number,line)[1])+']) # number='+ re.sub("\D","",re.findall(number_1,line)[0])+"\n"
    return sentence,total_number

def x2ix(tab,line,total_number):
    sentence= tab+"prog.i(input_qubit["+ re.sub("\D","",re.findall(number,line)[0])+ ']) # number='+ re.sub("\D","",re.findall(number_1,line)[0])+"\n"+\
            tab+"prog.x(input_qubit["+ re.sub("\D","",re.findall(number,line)[0])+ ']) # number='+ str(total_number+1) +"\n"
    return sentence,total_number+1

def x2hzh(tab,line,total_number):
    sentence= tab+"prog.h(input_qubit["+ re.sub("\D","",re.findall(number,line)[0])+ ']) # number='+ re.sub("\D","",re.findall(number_1,line)[0])+"\n"+\
                tab+"prog.z(input_qubit["+ re.sub("\D","",re.findall(number,line)[0])+ ']) # number='+ str(total_number+1) +"\n"+\
                tab+"prog.h(input_qubit["+ re.sub("\D","",re.findall(number,line)[0])+ ']) # number='+ str(total_number+2) +"\n"
    return sentence,total_number+2

def z2cx(tab,line,total_number):
    
    currentnumber= re.sub("\D","",re.findall(number,line)[0])
    if int(currentnumber)==0:
        added_number=1
    else:
        added_number=0
    sentence= tab+"prog.cx(input_qubit["+ str(currentnumber) + '],input_qubit['+ str(added_number)+']) # number='+ re.sub("\D","",re.findall(number_1,line)[0])+"\n"+tab+"prog.z(input_qubit["+  str(currentnumber)+ ']) # number='+ str(total_number+1) +"\n"+tab+"prog.cx(input_qubit["+ str(currentnumber) + '],input_qubit['+str(added_number)+']) # number='+ str(total_number+2) +"\n"
    return sentence,total_number+2
    
def x2cx(tab,line,total_number):
    currentnumber= re.sub("\D","",re.findall(number,line)[0])
    if int(currentnumber)==0:
        added_number=1
    else:
        added_number=0

    sentence= tab+"prog.cx(input_qubit["+ str(added_number) + '],input_qubit['+ str(currentnumber)+']) # number='+ re.sub("\D","",re.findall(number_1,line)[0])+"\n"+tab+"prog.x(input_qubit["+  str(currentnumber)+ ']) # number='+ str(total_number+1) +"\n"+tab+"prog.cx(input_qubit["+ str(added_number) + '],input_qubit['+str(currentnumber)+']) # number='+ str(total_number+2) +"\n"
    return sentence,total_number+2

def cz2h(tab,line,total_number):
    sentence= tab+"prog.h(input_qubit["+ re.sub("\D","",re.findall(number,line)[1])+']) # number='+ re.sub("\D","",re.findall(number_1,line)[0])+"\n"+\
            tab+"prog.cx(input_qubit["+ re.sub("\D","",re.findall(number,line)[0])+ '],input_qubit['+ re.sub("\D","",re.findall(number,line)[1])+']) # number='+ str(total_number+1) +"\n"+\
                tab+"prog.h(input_qubit["+ re.sub("\D","",re.findall(number,line)[1])+']) # number='+ str(total_number+2) +"\n"
    return sentence,total_number+2

def cx2h(tab,line,total_number):
    sentence= tab+"prog.h(input_qubit["+ re.sub("\D","",re.findall(number,line)[1])+']) # number='+ re.sub("\D","",re.findall(number_1,line)[0])+"\n"+\
            tab+"prog.cz(input_qubit["+ re.sub("\D","",re.findall(number,line)[0])+ '],input_qubit['+ re.sub("\D","",re.findall(number,line)[1])+']) # number='+ str(total_number+1) +"\n"+\
                tab+"prog.h(input_qubit["+ re.sub("\D","",re.findall(number,line)[1])+']) # number='+ str(total_number+2) +"\n"
    return sentence,total_number+2


In [ ]:

def simulator_to_state_vector (file_in):  # change the backend of seed program to state_vector
    find_backend=re.compile("qasm_simulator")
    change_file_name=re.compile("startQiskit[0-9]+")
    readfile = open(file_in)
    file_out_name= "startQiskit_statevector_"+ re.sub("\D","",file_in)+".py"
    measure_find=re.compile("measure_all")
    writefile = open(file_out_name,"w")
    line = readfile.readline()
    while line:
        if measure_find.search(line):
            line = readfile.readline()
            continue
        if find_backend.search(line):
            cc=re.sub(find_backend,"statevector_simulator",line)
            writefile.write(cc)
            line = readfile.readline()
            continue
        if change_file_name.search(line):
            number=re.sub("\D","",re.findall(change_file_name,line)[0] )
            cc1=re.sub(change_file_name,"startQiskit_statevector_"+str(number),line)
            writefile.write(cc1)
        else:
            writefile.write(line)
        line = readfile.readline()
    readfile.close()
    writefile.close()
    return file_out_name
    
def simulator_to_noise (file_in):   # change the backend of seed program to noise (fakeYorktown or Fake Viego)
    find_backend=re.compile("backend = ")
    change_file_name=re.compile("startQiskit[0-9]+")
    readfile = open(file_in)
    file_out_name= "startQiskit_noise_"+ re.sub("\D","",file_in)+".py"
    measure_find=re.compile("measure_all")
    writefile = open(file_out_name,"w")
    line = readfile.readline()
    while line:
        
        if find_backend.search(line):
            writefile.write("    backend = FakeYorktown()")
            line = readfile.readline()
            continue
        if change_file_name.search(line):
            number=re.sub("\D","",re.findall(change_file_name,line)[0] )
            cc1=re.sub(change_file_name,"startQiskit_noise_"+str(number),line)
            writefile.write(cc1)
        else:
            writefile.write(line)
        line = readfile.readline()
    readfile.close()
    writefile.close()
    return file_out_name
    
def simulator_to_realIBM (file_in):  #change the backend to a real IBM quantum computer
    find_backend=re.compile("backend = ")
    change_file_name=re.compile("startQiskit[0-9]+")
    readfile = open(file_in)
    file_out_name= "startQiskit_real_"+ re.sub("\D","",file_in)+".py"
    measure_find=re.compile("measure_all")
    writefile = open(file_out_name,"w")
    line = readfile.readline()
    while line:
        
        if find_backend.search(line):
            writefile.write("    IBMQ.load_account() \n"
                            "    provider = IBMQ.get_provider(hub='ibm-q') \n"
                            "    provider.backends()\n")
            writefile.write("    backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 2 and "
                            "not x.configuration().simulator and x.status().operational == True))\n")
            line = readfile.readline()
            continue
        if change_file_name.search(line):
            number=re.sub("\D","",re.findall(change_file_name,line)[0] )
            cc1=re.sub(change_file_name,"startQiskit_real_"+str(number),line)
            writefile.write(cc1)
        else:
            writefile.write(line)
        line = readfile.readline()
    readfile.close()
    writefile.close()
    return file_out_name



In [4]:


def generate_add(tab,qubit_number,total_number):   #####create a sentence which uses for adding method 
    gate_set_qiskit = ["prog.h","prog.x","prog.y","prog.z","prog.rx","prog.cx","prog.swap"]
    q_number=random.randint(0,qubit_number-1)
    gatename=random.choice(gate_set_qiskit)
#     gatename="prog.cx"
    if gatename=="prog.h" or gatename=="prog.x" or gatename=="prog.y" or gatename=="prog.z":
        sentence=tab+gatename+"(input_qubit["  + str(q_number) +"])"+ " # number="+ str(total_number+1)
    elif gatename=="prog.rx":               ####for rx gate, it should add a random angle.
        sentence=tab+gatename+"(" + str((random.randint(-1000,1000))/1000 *pi) +",input_qubit["  + str(q_number) +"])"+ " # number="+ str(total_number+1) 
    elif gatename== "prog.cx" or gatename== "prog.swap":
        q1=random.randint(1,qubit_number-1)
        q2=random.randint(1,qubit_number-1)
        while q1==q2:
            q2=random.randint(1,qubit_number-1)
        if qubit_number==2:
            q1=0
            q2=1
        sentence=tab+gatename+  "(input_qubit[" + str(q1)+  "],input_qubit[" + str(q2)+ "]) # number=" + str(total_number+1)
    return sentence

def generate_change(tab,qubit_number,total_number,line):###ues the sentence which is gengerated by "generate_add" to nutateh seed program and save the new file as a new .py file 
    single_gate_h = re.compile("prog.h")
    single_gate_x = re.compile("prog.x")
    single_gate_y = re.compile("prog.y")
    single_gate_z = re.compile("prog.z")
    
    double_gate_cx = re.compile("prog.cx")
    double_gate_cy = re.compile("prog.cy")
    double_gate_cz = re.compile("prog.cz")
    double_gate_swap= re.compile("prog.swap")
    
    rotate_gate_rx = re.compile("prog.rx")
    rotate_gate_ry = re.compile("prog.ry")
    rotate_gate_rz = re.compile("prog.rz")
    number= "\[[0-9]+\]"
    number_1=" # number=[0-9]+"
    if single_gate_h.search(line):
        gate=random.choice(["x","y","z"])
        sentence=tab+"prog."+gate+"(input_qubit["  + re.sub("\D","",re.findall(number,line)[0]) +"])"+ " # number="+ re.sub("\D","",re.findall(number_1,line)[0])
    if single_gate_x.search(line):
        gate=random.choice(["h","y","z"])
        sentence=tab+"prog."+gate+"(input_qubit["  + re.sub("\D","",re.findall(number,line)[0]) +"])"+ " # number="+ re.sub("\D","",re.findall(number_1,line)[0])
    if single_gate_y.search(line):
        gate=random.choice(["x","h","z"])
        sentence=tab+"prog."+gate+"(input_qubit["  + re.sub("\D","",re.findall(number,line)[0]) +"])"+ " # number="+ re.sub("\D","",re.findall(number_1,line)[0])
    if single_gate_z.search(line):
        gate=random.choice(["x","y","h"])
        sentence=tab+"prog."+gate+"(input_qubit["  + re.sub("\D","",re.findall(number,line)[0]) +"])"+ " # number="+ re.sub("\D","",re.findall(number_1,line)[0])
        
        
    if double_gate_cx.search(line):
        gate=random.choice(["cy","cz"])
        sentence=tab+"prog."+gate+  "(input_qubit[" + re.sub("\D","",re.findall(number,line)[0])+  "],input_qubit[" + re.sub("\D","",re.findall(number,line)[1])+ "]) # number=" + re.sub("\D","",re.findall(number_1,line)[0])
    if double_gate_cy.search(line):
        gate=random.choice(["cx","cz"])
        sentence=tab+"prog."+gate+  "(input_qubit[" + re.sub("\D","",re.findall(number,line)[0])+  "],input_qubit[" + re.sub("\D","",re.findall(number,line)[1])+ "]) # number=" + re.sub("\D","",re.findall(number_1,line)[0])
    if double_gate_cz.search(line):
        gate=random.choice(["cy","cx"])
        sentence=tab+"prog."+gate+  "(input_qubit[" + re.sub("\D","",re.findall(number,line)[0])+  "],input_qubit[" + re.sub("\D","",re.findall(number,line)[1])+ "]) # number=" + re.sub("\D","",re.findall(number_1,line)[0])
    if double_gate_swap.search(line):
        q1=random.randint(1,qubit_number-1)
        while q1==re.sub("\D","",re.findall(number,line)[0]):
            q1=random.randint(1,qubit_number-1)
        
        q2=random.randint(1,qubit_number-1)
        while q1==q2:
            q2=random.randint(1,qubit_number-1)
        sentence=tab+"prog.swap(input_qubit[" + str(q1)+  "],input_qubit[" + str(q2)+ "]) # number=" + re.sub("\D","",re.findall(number_1,line)[0])
    
    if rotate_gate_rx.search(line):
        gate=random.choice(["ry","rz"])
        sentence=tab+"prog."+gate+"("+str((random.randint(-1000,1000))/1000 *pi)+",input_qubit["+ re.sub("\D","",re.findall(number,line)[0]) +"])"+ " # number="+ re.sub("\D","",re.findall(number_1,line)[0])
    if rotate_gate_ry.search(line):
        gate=random.choice(["rx","rz"])
        sentence=tab+"prog."+gate+"("+str((random.randint(-1000,1000))/1000 *pi)+",input_qubit["+ re.sub("\D","",re.findall(number,line)[0]) +"])"+ " # number="+ re.sub("\D","",re.findall(number_1,line)[0])
    if rotate_gate_rz.search(line):
        gate=random.choice(["ry","rx"])
        sentence=tab+"prog."+gate+"("+str((random.randint(-1000,1000))/1000 *pi)+",input_qubit["+ re.sub("\D","",re.findall(number,line)[0]) +"])"+ " # number="+ re.sub("\D","",re.findall(number_1,line)[0])
    return sentence

def exe_adding(add_sentence,add_place_number,totalnumber,inpyname,outpyname): ###ues the sentence which is gengerated by "generate_add" to nutateh seed program and save the new file as a new .py file 
    readfile = open(inpyname)
    writefile = open(outpyname,"w")   #set the name of new py file
    line = readfile.readline()
    total_number = re.compile("# total number=")
    add_place=re.compile("# number="+str(add_place_number) )

    wirte_new_file_name=re.compile("open" )
   
    while line:

        if add_place.search(line):
            writefile.write(line+add_sentence+"\n")
            line = readfile.readline()
            continue
        if wirte_new_file_name.search(line):
            writefile.write(re.sub("startQiskit[0-9]+",outpyname.split('.')[0],line)) ##change the csv name in the py files
            line = readfile.readline()
            continue
            
        if total_number.search(line):
            writefile.write("# total number="+str(totalnumber+1)+"\n")
        else:
            writefile.write(line)
        line = readfile.readline()
        
    readfile.close()
    writefile.close()
    
def exe_changing(sentence,place_number,inpyname,outpyname):  ###ues the sentence which is gengerated by "generate_change" to nutateh seed program and save the new file as a new .py file 
    readfile = open(inpyname)
    writefile = open(outpyname,"w")
    line = readfile.readline()
    change_place=re.compile("# number="+str(place_number) )
    wirte_new_file_name=re.compile("open" )
    while line:
        if wirte_new_file_name.search(line):
            writefile.write(re.sub("startQiskit[0-9]+",outpyname.split('.')[0],line))
#             print(re.sub(wirte_new_file_name,outpyname.split('.')[0],line))
            line = readfile.readline()
            continue
            
        if change_place.search(line):
            writefile.write(sentence+"\n")
        else:
            writefile.write(line)
        line = readfile.readline()
        
    readfile.close()
    writefile.close()

def exe_deleting(totalnumber,place_number,inpyname,outpyname): ##find the position which shou be delete and delete the line, then change the totoal number and save as a new file
    readfile = open(inpyname)
    writefile = open("temp","w")
    line = readfile.readline()
    change_place=re.compile("# number="+str(place_number) )
    wirte_new_file_name=re.compile( "open" )
    total_number = re.compile("# total number=")
    
    while line:
        
        if total_number.search(line):
            writefile.write("# total number="+str(totalnumber-1)+"\n")
            line = readfile.readline()
            continue
        if wirte_new_file_name.search(line):
            writefile.write(re.sub("startQiskit[0-9]+",outpyname.split('.')[0],line))
            line = readfile.readline()
            continue
        if change_place.search(line):
            writefile.write("")
        else:
            writefile.write(line)
        line = readfile.readline()
        
    readfile.close()
    writefile.close()
    
    readfile_1 = open('temp')
    writefile_1 = open(outpyname,"w")
    number_patern=re.compile("# number=")
    i=1
    line_1 = readfile_1.readline()
    while line_1:
        if number_patern.search(line_1):
            writefile_1.write(re.sub( "# number=[0-9]+","# number="+str(i),line_1))
            i +=1 
        else:
            writefile_1.write(line_1)
        line_1 = readfile_1.readline()
    readfile_1.close()
    writefile_1.close()
    os.remove("./temp")
    
def mute(file_in,file_out):  ###### this function includes the formal functions to mutate the seed program to a new seed program
    qubit_number_patter = re.compile("# qubit number=")
    operation_id = re.compile("# number=")
    nu = re.compile("# number=[0-9]+")
    total_operation_id = re.compile("# total number=")
    qubit_number = 0
    total_number = 0
    mutation_number = 1
    flag = 0      
    gate_set_qiskit = ["prog.h","prog.x","prog.y","prog.z","prog.rx","prog.cx","prog.swap"]
    readfile = open(file_in)
    line = readfile.readline()
    
    while line:
        if qubit_number_patter.search(line):
            qubit_number= int(re.sub("\D","",line))
        if total_operation_id.search(line):
            total_number=int(re.sub("\D","",line))
            if total_number==1:
                mutation_number = 1
            else:
                mutation_number=random.randint(1,total_number-1)   #mutation_number randomly decides which line to use for mutation 
        if operation_id.search(line):
#             print(re.findall(operation_id,line))
            flag=int( re.sub("\D","",re.findall(nu,line)[0]))   ##uesd the find the line which is decided by mutation_number
            
        if flag == mutation_number:    # read each line of the file , and adopt different operation for each line
            if find_tab(line) != "":
                tab=find_tab(line)
            if total_number != 1:                 ####randomly chooes add, delete or change operation
                muta_choose=random.randint(1,3)
            else:
                muta_choose=random.randint(1,2)
#             muta_choose=2
            if muta_choose == 1:
                print("adding")
                resu=generate_add(tab,qubit_number,total_number)
                exe_adding(resu,mutation_number,total_number,file_in,file_out)
                readfile.close()
                return 1
            elif muta_choose == 2:
                print("changing")
#                 print(mutation_number)
                resu=generate_change(tab,qubit_number,total_number,line)
#                 print(file_in)
#                 print(file_out)
                exe_changing(resu,mutation_number,file_in,file_out)
                
                readfile.close()
                return 1
            else:
                print("delecting")
                exe_deleting(total_number,mutation_number,file_in,file_out)
                readfile.close()
                return 1
        line = readfile.readline()
    
    readfile.close()
    return 0

In [5]:
def readcir(file_in):    #this function is uesd to read the circuit in each csv  and change it to string 
    with open(file_in, 'r',encoding="utf-8") as f:
        a=f.readlines()
        circut=""
        for i in range(4,len(a)):
            circut += a[i]
    return circut

In [6]:
def child_loop():
    flag=210               #####this loop is uesd to generate euqivalent circuits and change the backend of them
    pipei=re.compile("startQiskit[0-9]+")
    while flag<232:             ####### this number minus the initial number of flag is the number of equivalent circuits of each seed program
        wait=[]
        for file in os.listdir("."):
            if pipei.search(file):
                wait.append(file)
        choose_file=random.choice(wait)
        times=0
        while times<10:
            bb=makeeuqalfiles(choose_file,"startQiskit"+str(flag)+".py")
            if bb==0:
                bb=makeeuqalfiles(choose_file,"startQiskit"+str(flag)+".py")
                times +=1
            else:
                break
        name=simulator_to_state_vector("startQiskit"+str(flag)+".py")

        execution(name)
        circuit=readcir(name.split(".")[0]+".csv")
        if circuit not in cir:           ########detect if the circuit is repeated, if so, delete the current circuit and recreate another one.
            cir.append(circuit)
            simulator_to_realIBM("startQiskit"+str(flag)+".py")
            simulator_to_noise("startQiskit"+str(flag)+".py")
            print("generate from"+str(choose_file)+"  to  "+str(flag))
            flag += 1
        else:
            os.remove("startQiskit"+str(flag)+".py")
            os.remove(name)
            os.remove(name.split(".")[0]+".csv")
        



In [7]:
%%time
whole_loop=1
cir=[]
py_file=re.compile("start")
####this is the main function of system.
try:
    while whole_loop<100:     ###ues this number to control the numbers of mutation

        seed_choose=  random.choice(os.listdir("seed") )
        mute( "seed/"+seed_choose,"startQiskit"+str(whole_loop)+".py")
        seed_name=simulator_to_state_vector("startQiskit"+str(whole_loop)+".py")
        execution(seed_name)
        seed_cir=readcir(seed_name.split(".")[0]+".csv")
        if seed_cir in cir:     ########detect if the circuit is repeated, if so, delete the current circuit and recreate another one.
            os.remove("startQiskit"+str(whole_loop)+".py")
            os.remove(seed_name)
            os.remove(seed_name.split(".")[0]+".csv")
        else:        ######if the circuit is not repeated ,then ues it as a new seed program to generate equivalent circuits.
            shutil.copy("startQiskit"+str(whole_loop)+".py","./seed")
            cir.append(seed_cir)
            simulator_to_realIBM( "startQiskit"+str(whole_loop)+".py" )
            simulator_to_noise( "startQiskit"+str(whole_loop)+".py" )
            print("generate seed file from"+str(seed_choose)+"  to  "+str(whole_loop))
            print("---------------------------------------------------")
            child_loop()    
            if os.path.exists(str(whole_loop)) is True:   ######after each iteration, move all the generated files to a new folder  for further calculation
                shutil.rmtree(str(whole_loop))
            os.mkdir(str(whole_loop) )
            files=os.listdir(".")
            for file in files:
                if py_file.search(file) :
                    shutil.move(str(file),str(whole_loop))
            whole_loop += 1
except Exception as e:
    print("OS error:" + str(e))

changing
generate seed file fromstartQiskit0.py  to  1
---------------------------------------------------
generate fromstartQiskit1.py  to  210
generate fromstartQiskit210.py  to  211
generate fromstartQiskit1.py  to  212
generate fromstartQiskit1.py  to  213
generate fromstartQiskit212.py  to  214
generate fromstartQiskit213.py  to  215
generate fromstartQiskit213.py  to  216
generate fromstartQiskit1.py  to  217
generate fromstartQiskit214.py  to  218
generate fromstartQiskit211.py  to  219
generate fromstartQiskit217.py  to  220
generate fromstartQiskit213.py  to  221
generate fromstartQiskit213.py  to  222
generate fromstartQiskit216.py  to  223
generate fromstartQiskit217.py  to  224
generate fromstartQiskit1.py  to  225
generate fromstartQiskit225.py  to  226
generate fromstartQiskit221.py  to  227
generate fromstartQiskit1.py  to  228
generate fromstartQiskit215.py  to  229
generate fromstartQiskit213.py  to  230
generate fromstartQiskit221.py  to  231
adding
generate seed file

generate fromstartQiskit212.py  to  217
generate fromstartQiskit216.py  to  218
generate fromstartQiskit211.py  to  219
generate fromstartQiskit212.py  to  220
generate fromstartQiskit213.py  to  221
generate fromstartQiskit219.py  to  222
generate fromstartQiskit215.py  to  223
generate fromstartQiskit215.py  to  224
generate fromstartQiskit224.py  to  225
generate fromstartQiskit215.py  to  226
generate fromstartQiskit211.py  to  227
generate fromstartQiskit210.py  to  228
generate fromstartQiskit217.py  to  229
generate fromstartQiskit219.py  to  230
generate fromstartQiskit230.py  to  231
delecting
generate seed file fromstartQiskit7.py  to  10
---------------------------------------------------
generate fromstartQiskit10.py  to  210
generate fromstartQiskit10.py  to  211
generate fromstartQiskit211.py  to  212
generate fromstartQiskit211.py  to  213
generate fromstartQiskit210.py  to  214
generate fromstartQiskit211.py  to  215
generate fromstartQiskit215.py  to  216
generate from

generate fromstartQiskit215.py  to  226
generate fromstartQiskit212.py  to  227
generate fromstartQiskit213.py  to  228
generate fromstartQiskit218.py  to  229
generate fromstartQiskit221.py  to  230
generate fromstartQiskit219.py  to  231
adding
generate seed file fromstartQiskit0.py  to  18
---------------------------------------------------
generate fromstartQiskit18.py  to  210
generate fromstartQiskit18.py  to  211
generate fromstartQiskit18.py  to  212
generate fromstartQiskit210.py  to  213
generate fromstartQiskit210.py  to  214
generate fromstartQiskit213.py  to  215
generate fromstartQiskit214.py  to  216
generate fromstartQiskit211.py  to  217
generate fromstartQiskit214.py  to  218
generate fromstartQiskit217.py  to  219
generate fromstartQiskit214.py  to  220
generate fromstartQiskit212.py  to  221
generate fromstartQiskit210.py  to  222
generate fromstartQiskit210.py  to  223
generate fromstartQiskit210.py  to  224
generate fromstartQiskit215.py  to  225
generate fromstar

generate fromstartQiskit26.py  to  210
generate fromstartQiskit210.py  to  211
generate fromstartQiskit26.py  to  212
generate fromstartQiskit212.py  to  213
generate fromstartQiskit210.py  to  214
generate fromstartQiskit26.py  to  215
generate fromstartQiskit213.py  to  216
generate fromstartQiskit216.py  to  217
generate fromstartQiskit210.py  to  218
generate fromstartQiskit215.py  to  219
generate fromstartQiskit211.py  to  220
generate fromstartQiskit211.py  to  221
generate fromstartQiskit218.py  to  222
generate fromstartQiskit214.py  to  223
generate fromstartQiskit216.py  to  224
generate fromstartQiskit216.py  to  225
generate fromstartQiskit212.py  to  226
generate fromstartQiskit218.py  to  227
generate fromstartQiskit221.py  to  228
generate fromstartQiskit226.py  to  229
generate fromstartQiskit210.py  to  230
generate fromstartQiskit214.py  to  231
delecting
generate seed file fromstartQiskit10.py  to  27
---------------------------------------------------
generate from

generate fromstartQiskit214.py  to  218
generate fromstartQiskit211.py  to  219
generate fromstartQiskit212.py  to  220
generate fromstartQiskit219.py  to  221
generate fromstartQiskit34.py  to  222
generate fromstartQiskit210.py  to  223
generate fromstartQiskit214.py  to  224
generate fromstartQiskit224.py  to  225
generate fromstartQiskit215.py  to  226
generate fromstartQiskit216.py  to  227
generate fromstartQiskit221.py  to  228
generate fromstartQiskit213.py  to  229
generate fromstartQiskit228.py  to  230
generate fromstartQiskit218.py  to  231
changing
generate seed file fromstartQiskit20.py  to  35
---------------------------------------------------
generate fromstartQiskit35.py  to  210
generate fromstartQiskit210.py  to  211
generate fromstartQiskit211.py  to  212
generate fromstartQiskit35.py  to  213
generate fromstartQiskit210.py  to  214
generate fromstartQiskit214.py  to  215
generate fromstartQiskit215.py  to  216
generate fromstartQiskit210.py  to  217
generate froms

...

generate fromstartQiskit217.py  to  219
generate fromstartQiskit215.py  to  220
generate fromstartQiskit214.py  to  221
generate fromstartQiskit221.py  to  222
generate fromstartQiskit214.py  to  223
generate fromstartQiskit223.py  to  224
generate fromstartQiskit212.py  to  225
generate fromstartQiskit210.py  to  226
generate fromstartQiskit214.py  to  227
generate fromstartQiskit59.py  to  228
generate fromstartQiskit220.py  to  229
generate fromstartQiskit216.py  to  230
generate fromstartQiskit211.py  to  231
changing
generate seed file fromstartQiskit36.py  to  60
---------------------------------------------------
generate fromstartQiskit60.py  to  210
generate fromstartQiskit60.py  to  211
generate fromstartQiskit210.py  to  212
generate fromstartQiskit211.py  to  213
generate fromstartQiskit60.py  to  214
generate fromstartQiskit212.py  to  215
generate fromstartQiskit213.py  to  216
generate fromstartQiskit215.py  to  217
generate fromstartQiskit214.py  to  218
generate fromst

generate fromstartQiskit218.py  to  227
generate fromstartQiskit210.py  to  228
generate fromstartQiskit213.py  to  229
generate fromstartQiskit67.py  to  230
generate fromstartQiskit220.py  to  231
delecting
generate seed file fromstartQiskit38.py  to  68
---------------------------------------------------
generate fromstartQiskit68.py  to  210
generate fromstartQiskit68.py  to  211
generate fromstartQiskit210.py  to  212
generate fromstartQiskit211.py  to  213
generate fromstartQiskit213.py  to  214
generate fromstartQiskit213.py  to  215
generate fromstartQiskit211.py  to  216
generate fromstartQiskit216.py  to  217
generate fromstartQiskit216.py  to  218
generate fromstartQiskit212.py  to  219
generate fromstartQiskit68.py  to  220
generate fromstartQiskit218.py  to  221
generate fromstartQiskit218.py  to  222
generate fromstartQiskit68.py  to  223
generate fromstartQiskit221.py  to  224
generate fromstartQiskit212.py  to  225
generate fromstartQiskit221.py  to  226
generate fromst

generate fromstartQiskit76.py  to  211
generate fromstartQiskit211.py  to  212
generate fromstartQiskit211.py  to  213
generate fromstartQiskit211.py  to  214
generate fromstartQiskit210.py  to  215
generate fromstartQiskit211.py  to  216
generate fromstartQiskit215.py  to  217
generate fromstartQiskit216.py  to  218
generate fromstartQiskit216.py  to  219
generate fromstartQiskit216.py  to  220
generate fromstartQiskit217.py  to  221
generate fromstartQiskit76.py  to  222
generate fromstartQiskit217.py  to  223
generate fromstartQiskit215.py  to  224
generate fromstartQiskit221.py  to  225
generate fromstartQiskit210.py  to  226
generate fromstartQiskit221.py  to  227
generate fromstartQiskit218.py  to  228
generate fromstartQiskit218.py  to  229
generate fromstartQiskit214.py  to  230
generate fromstartQiskit212.py  to  231
adding
generate seed file fromstartQiskit22.py  to  77
---------------------------------------------------
generate fromstartQiskit77.py  to  210
generate fromsta

generate fromstartQiskit216.py  to  220
generate fromstartQiskit84.py  to  221
generate fromstartQiskit213.py  to  222
generate fromstartQiskit215.py  to  223
generate fromstartQiskit211.py  to  224
generate fromstartQiskit222.py  to  225
generate fromstartQiskit216.py  to  226
generate fromstartQiskit223.py  to  227
generate fromstartQiskit212.py  to  228
generate fromstartQiskit212.py  to  229
generate fromstartQiskit218.py  to  230
generate fromstartQiskit217.py  to  231
adding
generate seed file fromstartQiskit83.py  to  85
---------------------------------------------------
generate fromstartQiskit85.py  to  210
generate fromstartQiskit210.py  to  211
generate fromstartQiskit211.py  to  212
generate fromstartQiskit85.py  to  213
generate fromstartQiskit212.py  to  214
generate fromstartQiskit211.py  to  215
generate fromstartQiskit215.py  to  216
generate fromstartQiskit210.py  to  217
generate fromstartQiskit217.py  to  218
generate fromstartQiskit211.py  to  219
generate fromsta

generate fromstartQiskit224.py  to  228
generate fromstartQiskit214.py  to  229
generate fromstartQiskit224.py  to  230
generate fromstartQiskit226.py  to  231
adding
generate seed file fromstartQiskit89.py  to  93
---------------------------------------------------
generate fromstartQiskit93.py  to  210
generate fromstartQiskit210.py  to  211
generate fromstartQiskit210.py  to  212
generate fromstartQiskit212.py  to  213
generate fromstartQiskit211.py  to  214
generate fromstartQiskit93.py  to  215
generate fromstartQiskit213.py  to  216
generate fromstartQiskit93.py  to  217
generate fromstartQiskit214.py  to  218
generate fromstartQiskit93.py  to  219
generate fromstartQiskit211.py  to  220
generate fromstartQiskit211.py  to  221
generate fromstartQiskit220.py  to  222
generate fromstartQiskit215.py  to  223
generate fromstartQiskit210.py  to  224
generate fromstartQiskit218.py  to  225
generate fromstartQiskit224.py  to  226
generate fromstartQiskit93.py  to  227
generate fromstart